# Log Parse D3

This page shows ...

#### Setup the environment

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import requests
import itertools
import os.path
from numpngw import write_png

#### Get a list of remote CamHD files to process

In [2]:
filenames = []
with open('d3.txt') as f:
    for line in f:
        filenames.append(line.strip())

#### Get the timestamp from each video file

In [3]:
#only do this once
#movtimes = []
#for filename in filenames:
#    try:
#        movtimes.append(camhd.get_timestamp(filename))
#    except:
#        print(filename)

# now just read it from a file
movtimes = []
with open('d3_movtimes.csv', 'r') as csvfile:
    for line in csvfile:
        movtimes.append(line.split(',')[1].strip())

#### Write start times to file

In [4]:
#with open('d4_movtimes.csv', 'w') as csvfile:
#    for i, filename in enumerate(filenames):
#        csvfile.write('%s,%s\n' % (filename, movtimes[i]))

#### Get list of log files (from local)

In [5]:
#%%time
#logfiles = []
#for i,filename in enumerate(filenames):
#    logfile = '/data/camhda301_%s_%s.log' % (filename[96:102], filename[103:107])
#    if os.path.isfile(logfile):
#        logfiles.append(logfile)

#### Get list of log file (from remote)

In [6]:
#%%time
#logfiles = []
#for i,filename in enumerate(filenames):
#    logfile = '%s/camhda301_%s_%s.log' % (filename[0:83], filename[96:102], filename[103:107])
#    r = requests.head(logfile)
#    if r.status_code==200:
#        logfiles.append(logfile)
        #continue
#    else:
#        print(logfile)
#        break
        
    #logfile = '%s/camhda301_%s_%s%s.log' % (filename[0:83], filename[96:102], filename[103:105], str(int(filename[105:107])-15))
    #r = requests.head(logfile)
    #print(logfile)
    #break
    #if r.status_code==200:
    #    print(logfile)
    #    logfiles.append(logfile)
    #else:
    #    logfiles.append(None)

#### Read logtimes.csv

In [7]:
logfiles = []
logtimes = []
with open('logtimes.csv', 'r') as f:
    for line in f:
        logfiles.append(line.split(',')[0])
        logtimes.append(line.split(',')[1].strip())

#### Build d3_meta

In [8]:
d3_meta = []
for i, filename in enumerate(filenames):
    for j, logtime in enumerate(logtimes):
        if int(movtimes[i]) > (int(logtime)-60) and int(movtimes[i]) < (int(logtime)+60):
            d3_meta.append([filename, str(movtimes[i]), '/data/%s' % logfiles[j], str(logtime)])

#### Get approximate time of p2z1 and append to d3_meta

In [9]:
def get_p2z1(logfile):
    p2z1 = None
    with open(logfile, 'r') as f:
        for line in f:
            if 'pan=143' in line or 'pan=145' in line:
                for line in f:
                    if 'time' in line[0:4]:
                        p2z1 = line.split(' ')[1].strip()
                        return p2z1

In [10]:
for entry in d3_meta:
    p2z1 = get_p2z1(entry[2])
    entry.append('p2z1')
    entry.append(str(p2z1));

#### Write out d3_meta

In [11]:
with open('d3_meta.csv', 'w') as f:
    for entry in d3_meta:
        f.write('%s,%s,%s,%s,%s,%s\n' % (entry[0],entry[1],entry[2],entry[3],entry[4],entry[5]))